# ASTR19 Pretty Picture Notebook

The goal of this notebook is to make a pretty picture from some simulated JWST astronomical data. We will create an RGB image and save it as a .png file.

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
from astropy.io import fits

In [ ]:
# define file names
fdata_F277W = "data_18/F277W_cutout.fits"
fdata_F356W = "data_18/F356W_cutout.fits"
fdata_F444W = "data_18/F444W_cutout.fits"

# open the fits files
hdu_2 = fits.open(fdata_F277W)
hdu_3 = fits.open(fdata_F356W)
hdu_4 = fits.open(fdata_F444W)

# show information about fits files
hdu_2.info()
hdu_3.info()
hdu_4.info()

In all files, the "primary" data is an image of size 2000x2000 encoded as floats (with 32 bits), and it is in the "row" No 0 of that data.

In [ ]:
# get the image data
data_2 = hdu_2[0].data
data_3 = hdu_3[0].data
data_4 = hdu_4[0].data

In [ ]:
# when in doubt, print out!

#### Have a look at the data

The data arrays are images, so let's have a look at the data.

In [ ]:
# First, plot the data as is
f = plt.figure(figsize=(7,7))
plt.imshow(data_2)
plt.colorbar()

#### Why does the image look blank?

We are again in a situation where a few points are extremely bright, and everything else looks pale in comparison. We need to rescale the data, but applying the log-scale won't work because some data points are negative! Even worse: some points are NaN (Not a Number) because there was a bug during measurement.

We will define a threshold that will correspond to some minimum level, and a maximum level, both dependent on the data.

Let's a script to do that.

In [ ]:
def rescale_image(data,vmin=0.5,vmax=100):
    # copy the data to avoid accidentaly overwriting it
    data_tmp = data.copy()

    # compute the mean value over all data, excluding NaN (Not a Number)
    m = np.nanmean(data_tmp)

    # any data points below vpmin is assigned the value vpmin
    # any data points above vpmax is assigned the value vpmax
    vpmin = vmin * m
    vpmax = vmax * m
    data_tmp[data_tmp < vpmin] = vpmin
    data_tmp[data_tmp > vpmax] = vpmax
    
    # replace all NaN by vpmin
    data_tmp = np.nan_to_num(data_tmp,nan=vpmin)

    # apply log-scale
    data_tmp = np.log10(data_tmp)
    return data_tmp

#### Now let's take a quick look at rescaled data

In [ ]:
f = plt.figure(figsize=(5,5))

data_res = rescale_image(data_2)

plt.imshow(data_res)

# Let's make a 3-color image from these data

In [ ]:
### Let's combine the data into a single object, where each layer will correspond to a color
data_all = np.zeros((data_2.shape[0],data_2.shape[1],3))

data_all[:,:,0] = data_2 #red is filter F277W
data_all[:,:,1] = data_3 #green is filter F356W
data_all[:,:,2] = data_4 #blue is filter F444W

# rescale the data so we can see features
# try different values of vmin and vmax!
data_all_res = rescale_image(data_all)

In [ ]:
print(np.shape(data_all_res))

#### Make a function to map data to [0,1] rgb values

Right now, data_all_res array contains values of intensity (in arbitrary units), but it needs to be mapped to [0,1] for RGB coloring. Let's define a function a function that does this automatically.

In [ ]:
def remap(data):
    #copy the data to avoid accidently overwriting it
    data_tmp = data.copy()

    dmin = data_tmp.min()
    dmax = data_tmp.max()
    return (data_tmp - dmin)/(dmax-dmin)

#### Create an RGB image that is nx * ny * 3 in size, where each image is either R, G or B

In [ ]:
rgb_image = np.zeros((data_2.shape[0],data_2.shape[1],3))

rgb_image = remap(data_all_res)

#### Plot the RGB image and save to a PNG

In [ ]:
f,ax = plt.subplots(1,1,figsize=(7,7))
ax.axis('off')
ax.imshow(rgb_image)

plt.savefig('jwst_image.png',bbox_inches='tight',pad_inches=0,dpi=600)

This image is simulated JWST data from before its launch. JWST was launched in 2021 and started collecting data in 2022. In the Final Project, you are using actual JWST data. You will be looking at distant structures of our Universe using the most powerfull telescope built by humankind.